In [1]:
import face_recognition
import cv2
import os
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import time 






# Loading Pickle File For Know_Faces Data






In [2]:
filename="/Users/sujith/Desktop/dp_project/Face-Recognition-Attendance-Projects-main/junior_data_230.pkl"
face_pkl_230=open(filename,"rb")
known_data_1=pkl.load(face_pkl_230)

In [3]:
filename="/Users/sujith/Desktop/dp_project/Face-Recognition-Attendance-Projects-main/junior_data_remain.pkl"
face_pkl_remain=open(filename,"rb")
known_data_2=pkl.load(face_pkl_remain)

In [4]:
for i in known_data_2:
    known_data_1.append(i)
    

In [5]:
known_data=known_data_1

In [6]:
len(known_data)

313







# Taking pictures from the camera  source and saving it in a directory .





In [7]:
import cv2
cam = cv2.VideoCapture(0)
# making the image count as zero initially

img_counter = 0
while True:
    ret, frame = cam.read()
    if not ret:
        print('failed to grab frame')
        break
    cv2.imshow('test', frame)
    #to get continuous live video feed from my laptops webcam
    k  = cv2.waitKey(1)
    # Here the programe waits for the key to be pressed and return the key code 
    
    # KEY CODE OF "ESC" is 27
    
    # KEY CODE OF "Space" is 32
    
    # SO , Therfore if the escape key is been pressed, the app will stop
    
    if k == 27:
        print('escape hit, closing the app')
        break
    # if the spacebar key is been pressed
    
    # screenshots will be taken
    
    elif k  == 32:
        
        # the format for storing the images scrreenshotted
        img_name ="capture"+str(img_counter)+".jpg"
        loc="/Users/sujith/Desktop/dp_project/Face-Recognition-Attendance-Projects-main/Captured/"+img_name
        # saves the image as a png file
        cv2.imwrite(loc, frame)
        print('screenshot taken')
        # the number of images automaticallly increases by 1
        img_counter += 1

# release the camera
cam.release()

# stops the camera window
cv2.destroyAllWindows() 

screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
escape hit, closing the app


In [7]:
import glob
import os

pic_all=glob.glob(os.path.join("/Users/sujith/Desktop/dp_project/Face-Recognition-Attendance-Projects-main/Captured", '*'))

In [8]:
pic_all

['/Users/sujith/Desktop/dp_project/Face-Recognition-Attendance-Projects-main/Captured/test.jpg']

In [52]:
import cv2

def adjust_brightness_contrast(image, brightness=0, contrast=0):
    # Apply brightness and contrast adjustments
    adjusted_image = cv2.convertScaleAbs(image, alpha=contrast, beta=brightness)
    return adjusted_image

def check_dullness(image):
    
    # Calculating the average pixel intensity of the image
    average_intensity = cv2.mean(image)[0]
    
    # Setting  a threshold for dullnessss
    threshold = 200  
    
    if average_intensity < threshold:
        return True
    else:
        return False
count=0
for image_path in pic_all:

    image = cv2.imread(image_path)

# Checking if the image is dull or not!!!
    if check_dullness(image):
    # Adjust brightness and contrast
        adjusted_image = adjust_brightness_contrast(image, brightness=10, contrast=1.1)
    #cv2.imshow('Original Image', image)
        cv2.imshow('Adjusted Image', adjusted_image)
        cv2.imwrite("/Users/sujith/Desktop/dp_project/Face-Recognition-Attendance-Projects-main/Captured/"+str(count)+".jpg",adjusted_image)
        count=count+1
    
    #cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("The image is not dull.")


In [ ]:
import cv2
import numpy as np
import face_recognition

# Load the input image
image = cv2.imread('/Users/sujith/Desktop/dp_project/Face-Recognition-Attendance-Projects-main/Captured/test.jpg')

# Detect face locations in the image
face_locations = face_recognition.face_locations(image)

# Adjust brightness and contrast for each detected face
for face_location in face_locations:
    top, right, bottom, left = face_location

    # Extract the face region
    face_image = image[top:bottom, left:right]

    # Increase brightness and contrast of the face image
    alpha = 1.2  # Brightness adjustment (1.0 means no change)
    beta = 25  # Contrast adjustment (0 means no change)
    adjusted_image = cv2.convertScaleAbs(face_image, alpha=alpha, beta=beta)

    # Replace the original face region with the adjusted image
    image[top:bottom, left:right] = adjusted_image

# Display the modified image
#cv2.imshow("Modified Image", image)
cv2.imwrite('/Users/sujith/Desktop/dp_project/Face-Recognition-Attendance-Projects-main/Captured/test.jpg',image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [65]:
known_names_1=os.listdir("/Users/sujith/Desktop/dp_project/Face-Recognition-Attendance-Projects-main/images_230")
#known_names_1.remove(".DS_Store")
known_names_2=os.listdir("/Users/sujith/Desktop/dp_project/Face-Recognition-Attendance-Projects-main/images_remain")
#known_name_2.remove(".DS_Store")

In [66]:
known_names=known_names_1+(known_names_2)
known_names.remove(".DS_Store")

In [67]:
len(known_data)

313

In [68]:
recognised_faces=[]





### DRAWING A BOX AROUND FACES RECOGNISED IN THE PICTURE AND OUTPUTTING THE RECOGNISED FACES IN ALL THE IMAGES






In [69]:
img_counter=0
for i in pic_all:
    #starting counter 
    a=time.time()
    img=cv2.imread(i)
    
    
    face_locations = face_recognition.face_locations(img)
    b=time.time()
    print("time taken to identify face locations "+ str(b-a))
    
    
    face_encodings = face_recognition.face_encodings(img, face_locations,num_jitters=6)
    c=time.time()
    print("time taken to encode faces "+str(c-b))
    print(" ")
#changing here    img_counter=0
    for face_encoding, face_location in zip(face_encodings, face_locations):
        # Try to match the face with the known face
        matches = face_recognition.compare_faces(known_data, face_encoding,tolerance=0.5)
        #name = "Unknown"

        # Find the best match
        face_distances = face_recognition.face_distance(known_data, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_names[best_match_index]
            recognised_faces.append(name)
            top, right, bottom, left = face_location
            cv2.rectangle(img, (left+2, top+2), (right+2, bottom+2), (255, 0, 0), 4)
            cv2.putText(img, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 0, 255), 2)
            cv2.imshow("out",img)
            img_name ="capture"+str(img_counter)+".jpg"
            saved="/Users/sujith/Desktop/dp_project/Face-Recognition-Attendance-Projects-main/Detected/"+img_name
            img_counter=img_counter+1
            cv2.imwrite(saved,img)
        

            
            
            
            
unique_faces=list(set(recognised_faces))
#print(recognised_faces)
print(" ")
print(" ")
print(unique_faces)

time taken to identify face locations 42.06313896179199
time taken to encode faces 3.184621810913086
 
time taken to identify face locations 41.507081031799316
time taken to encode faces 3.1353800296783447
 
 
 
['B22283.png', 'B22030.png', 'B22189.png', 'B22149.png', 'B22028.png', 'B22310.png', 'B22221.png', 'B22178.png', 'B22016.png', 'B22067.png', 'B22066.png', 'B22294.png', 'B22001.png', 'B22242.png', 'B22230.png', 'B22129.png', 'B22037.png', 'B22300.png', 'B22117.png', 'B22055.png', 'B22054.png', 'B22021.png', 'B22216.png', 'B22062.png', 'B22005.png', 'B22161.png']


In [70]:
len(recognised_faces)

49

In [71]:
freq = {}
for items in recognised_faces:
    freq[items] = recognised_faces.count(items)
    
freq

{'B22149.png': 2,
 'B22055.png': 2,
 'B22021.png': 2,
 'B22216.png': 2,
 'B22300.png': 2,
 'B22230.png': 2,
 'B22189.png': 2,
 'B22030.png': 2,
 'B22016.png': 2,
 'B22066.png': 2,
 'B22294.png': 2,
 'B22242.png': 2,
 'B22054.png': 2,
 'B22221.png': 2,
 'B22028.png': 2,
 'B22310.png': 2,
 'B22001.png': 2,
 'B22037.png': 2,
 'B22178.png': 2,
 'B22117.png': 2,
 'B22005.png': 2,
 'B22161.png': 2,
 'B22062.png': 2,
 'B22283.png': 1,
 'B22067.png': 1,
 'B22129.png': 1}

In [72]:
final_att={}
for i in freq.keys():
    if (freq[i]>0.5*(len(pic_all))):
        final_att[i]=freq[i]
    else:
        pass
    
    

In [56]:
final_att

{}

In [111]:
final_attendance=list(final_att.keys())
len(final_attendance)

17

 
 
 
 
 
 
 
 
# DETECTING ALL FACES IN THE FRAME 
 
 
 
 
 
 
 

In [23]:



for i in pic_all:
    #starting counter 
    a=time.time()
    img=cv2.imread(i)
    
    
    face_locations = face_recognition.face_locations(img)
    b=time.time()
    print("time taken to identify face locations "+ str(b-a))
    
    
    face_encodings = face_recognition.face_encodings(img, face_locations,num_jitters=5)
    c=time.time()
    print("time taken to encode faces "+str(c-b))
    print(" ")
    img_counter=0
    
    
    
    for face_encoding, face_location in zip(face_encodings, face_locations):
        # Try to match the face with the known faces
        matches = face_recognition.compare_faces(known_data, face_encoding,tolerance=0.5)
        #name = "Unknown"

        # Find the best match
        face_distances = face_recognition.face_distance(known_data, face_encoding)
        best_match_index = np.argmin(face_distances)
        
        
        #if matches[best_match_index]:
        name = known_names[best_match_index]
        
        
        recognised_faces.append(name)
        top, right, bottom, left = face_location
        cv2.rectangle(img, (left, top), (right, bottom), (0, 0, 255), 2)
        #cv2.putText(img, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 1)
        
        
        cv2.imshow("out",img)
        img_name ="capture"+str(img_counter)+".jpg"
        saved="/Users/sujith/Desktop/dp_project/Face-Recognition-Attendance-Projects-main/Detected/"+img_name
        img_counter=img_counter+1
        cv2.imwrite(saved,img)    

time taken to identify face locations 1.114548921585083
time taken to encode faces 1.14182710647583
 


In [36]:
recognised_faces

['aman_min.jpg', 'bhunia-min.jpg', 'sujith-min.jpg']

In [8]:
a=known_data

In [10]:
len(a)

29

In [11]:
len(a[0])

128

In [17]:
known_names

['alekya.jpeg',
 'simroop.jpeg',
 'Bipin.jpeg',
 'bhunia-min.jpg',
 'Aditi.jpg',
 'divyasree.jpeg',
 'gargi.jpeg',
 'Geetamsh.jpg',
 'Nitya.jpeg',
 'shravya.jpeg',
 'nikhil.jpg',
 'Yuraj.jpg',
 'soham.jpg',
 'sujith-min.jpg',
 'Paras.jpg',
 'anirudh.jpeg',
 'Aritra.jpeg',
 'Devidas.jpg',
 'charan.jpeg',
 'Anuj.jpeg',
 'Sanjay.jpg',
 'gaurav.jpeg',
 'garima.jpeg',
 'gopesh.jpeg',
 'Mayank.jpeg',
 'dhruv-min.jpg',
 'aman_min.jpg',
 'Piyush.jpg',
 'jatin.jpeg']

In [53]:
a=[1,2,3,4]